# Speeding Up with Cython

We should be able to use Cython to speed up our functions.

We should also be able to work in 8-bit space - we are not worried about being exact.

The only library we are using is numpy so we need to work out how to use Cython with Numpy. Here is an article - https://blog.paperspace.com/faster-numpy-array-processing-ndarray-cython/.

## Core Classes

We have three core classes:
* Covariance
* Power Iterator
* VPU

### Covariance

We'll start with the ZeroMeanCovarianceUnit.

Data:
* Size - small integer 8-bit unsigned.
* Count - we could limit to 255? (i.e. 8-bit unsigned) At the moment just runs upwards so would be large int.
* Square Sum - Numpy L * L 2D array
    * The data points x are 1D arrays of -1 to 1. 
    * x dot x.T results in an L by L matrix. Each i, j th entry is x_i * x_j. So min is -1 and max is 1. So x dot x.T is a 2D array which can be 3 bit (or two binary matrixes - one positive, one negative or one with abs and one with sign information).
    * Without scaling the square sum is a 2D integer array where the maximum integer is equal to the number of iterations.
    * If there are 256 iterations we go from 8-bit to 16-bit.
    * The covariance matrix is the square sum divided by the count. If we have 255 iterations each value could be mulitples of 1/255 - if we have 145 iterations it will be multiples of 1/145.
    
    
Can we have an assumed division in our covariance matrix and have it as modulo bit-length (e.g. 255)? NEED TO DEVELOP THIS. We have iteration batches of bit_length (e.g. 255). When we reach count = 256 we bit shift right (divide by 2) and add. Or if count is an 8-bit int when count = 255 we run and then reset count to 0. Hence, we have a fraction resolution set by bit_depth.

This will limit though to adapting to recent signals - we give our old signal and our new signal equal weight - but if our recent signal is very different but we have a normally stable variation (e.g. staring at a light) this will cause our covariance matrix to change. 

To cope we can change the right shift amounts. E.g. more heavy right shifts decrease our resolution of our updates but increase our resolution of our historic data. Weighting each.

A good question is do we need a 16-bit resolution or is a 8-bit resolution functional?

*Is this related to our scale system?* We have different covariance matrices representing different scales and then when we want the covariance matrix we combine and divide? Only need 10 log2 matrices. Buffers over time.

It is related to our time buffers and time series. The reconstruction is our covariance. See 2020-02-21 - Playing with Time Series Configurations.

Algorithm:
* On first run calculate SS as normal (or minus 0 - maybe minus 128)
* At iteration = bit_depth:
    * Add strata (our old buffer).
    * PBT the values between 0 and 255 to create binary version.
    * Add binary version to rolling sum for new strata.
    * If iteration = 255^i send estimate back and subtract instead of 128?
    
This is very very similar to our binary estimation case and reconstruction of predicted input. Coincidence?!

255^8 at 1Hz is 566 535 009 millennia. Even at 1MHz this would still be several millennia. (255^4)s in years is 133 years. CPU is 1200MHz. 1.2 * 10^9. for an approximate result, divide the time value by 3.154e+7

In [4]:
((255**8)/(1.2*(10**9)))/(3.154*(10**7))

472.36586735924993

In [8]:
(((2**8)**8)/(1.2*(10**9)))/(3.154*(10**7))

487.39019429585585

In [13]:
((2**64)/(1.2*(10**9)))/(3.154*(10**7))

487.39019429585585

In [5]:
((255**7)/(1.2*(10**9)))/(3.154*(10**7))

1.8524151661147055

So 8 sets of buffers would give us enough for 472 years at full CPU cycles.

In [1]:
512/4

128.0

Can we initialise the square sums as 128? Hence, we can subtract them? And change them?

Above 255 = 2^8 where 8 = bit depth so 8 stages at 255 is the same as 2^16. So the stages are 64/bit_depth with max 64.

In [15]:
bd = 68
print(bd)
if bd > 64: bd = 64
print(bd)
bd = 62
print(bd)
if bd > 64: bd = 64
print(bd)

68
64
62
62


Init if not 8 bit.
```
def __init__(self, size, bit_depth=8):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
            bit_depth: number of bits to store representations (multiple of 8) - max 64
        """
        self.size = size
        self.count = 0
        # Set bit_depth modulo 8
        if bit_depth > 64: bit_depth = 64
        if bit_depth < 8: bit_depth = 8
        stages = 64//bit_depth
        # Initialise Square Sum
        self.square_sums = np.ones(shape=(size, size, stages))
```

In [20]:
2**8

256

Does this only have benefit if we actually start to think that our representation space is limited?

When using Cython its often better to define lists as 1D numpy arrays. So let's start doing that.

```cdef np.array[double, dim=1] x_array```

In [22]:
import numpy as np
np.zeros(8)

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
# Can we do 2D PBT?
rand_vals_1 = np.random.randint(255, size=(4,4))
rand_vals_2 = np.random.randint(255, size=(4,4))
binary_values = np.where(rand_vals_1 > rand_vals_2, 1, 0)
print(rand_vals_1, rand_vals_2, binary_values)

[[127 183 235 207]
 [ 82 234 230  90]
 [178 115 193 226]
 [250 142 173 184]] [[171 228  30 215]
 [233 152 128 218]
 [141   9 252 124]
 [200  52 248  28]] [[0 0 1 0]
 [0 1 1 0]
 [1 1 0 1]
 [1 1 0 1]]


In [29]:
 binary_values[:, 0] = 128; binary_values

array([[128,   0,   1,   0],
       [128,   1,   1,   0],
       [128,   1,   0,   1],
       [128,   1,   0,   1]])

In [30]:
127//2

63

When ensuring that we remain within signed 8-bits integers we can divide by 2 before subtracting. But like our other residuals most of the energy is around the mean. So clipping maybe better. We thus clip at -64 and +63 subtract then times by 2.
```
# Right bit shift the current sum and the next stage sum
            # BUT LIKE OUR RESIDUALS WE MIGHT FIND ITS BETTER TO CLIP THAN DIVIDE
            current_stage = self.square_sum[:, :, 0]//2
            next_stage = self.square_sum[:, :, 1]//2
```

Clipping will only work if our input signal is already zero mean - 

In [ ]:
# Version with 8-bit bit-depth
class CovarianceUnit:
    """Variation where the mean is assumed to be 0."""
    
    def __init__(self, size):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
        """
        self.size = size
        # Set max value for signed int
        self.max_value = 126
        self.stages = 8
        # Initialise Square Sums 
        self.square_sum = np.zeros(shape=(size, size, stages), dtype=np.int8)
        # Set first set of square sums to -128
        self.square_sum[:, :, 0] = -128
        # Define counter for each stage
        self.stage_counter = np.zeros(stages, dtype=np.uint8)
        
    def update(self, x):
        """Add a data point x.
        
        This will involve a recursive check.

        x is a 1D numpy array of length 'size'.
        """
        # Increment current stage counter
        self.stage_counters[0] += 1
        # Add square of input array
        self.square_sum[:, :, 0] += np.dot(x, x.T)
        self.recursive_update(0)
                
    def recursive_update(i):
        """Update with recursive method.
        
        Args:
            i - stage to update - integer.
        """
        # Check i is within range
        if i > self.stages: 
            return
        # If i is within range check counter
        if self.stage_counters[i] >= 255:
            # Right bit shift the current sum and the next stage sum
            current_stage = self.square_sum[:, :, i]//2
            next_stage = self.square_sum[:, :, i+1]//2
            # Subtract estimate from next stage
            square_sum_dash = current_stage - next_stage
            # Clip at -64 and 63 then times by 2 - returns to int8 space
            square_sum_dash = np.clip(square_sum_dash, -63, 63)*2
            # PBT self.square_sum[:, :, 0] - BUT WILL THIS BE TERNARY? 
            rand_ints = np.random.randint(
                self.max_value, size=(self.size, self.size)
            )
            signs = np.signs(square_sum_dash)
            pbt_output = np.where(np.abs(square_sum_dash, rand_ints, 1, 0)
            # Add to next stage (with signs returned)
            self.square_sum[:, :, i+1] += pbt_output*signs
            # Reset the previous counter
            self.stage_counters[i] = 0
            # Increment next stage counter
            self.stage_counters[i+1] += 1
            self.recursive_update(i+1)
        else:
            return
                                  
    @property
    def covariance(self):
        """Compute covariance when requested."""
        # Reconstruct from square sums
        pass


# Power Iteration

We need to look at this next because we need to see how to return the covariance.

For power iterator we have the following data:
* ev - this is normally a float numpy 1D array with fractional values
* cov - this is normally a float numpy 2D array 
* scaler - this will have a fractional value greater than 1 (normally greater than 1.4).

We can keep ev and cov in signed 8 bit space and take the 1/127 out of the calculations (putting them at the front).

We have:
* cov^power \* ev - we can take out (1/127)^2 if power = 1.
* ev^T.input_data - we can take out (1/127) - input_data is -1, 0, 1 so so we have the same possible scale at ev but flipped - why we need symmetric scales - clip at -127 to 127.

We can either operate in the next power of 2 space - e.g. 16 bit vs 8 bit - the result is stored in 16 bit space to represent a max of 128 \* 128 (if signed - 255 if unsigned). OR. We can divide both the cov and the ev by 16 - so that the we have max 16\*16 = 256 (right shift by 3). 

In [ ]:
class PowerIterator:
    """Module to determine an eigenvector using power iteration."""

    def __init__(self, length):
        """Initialise.

        Args:
            length: integer setting the 1D size of the eigenvector.
        """
        # Initialise eigenvector as random vector - NOTE 8 bit
        # THIS WILL BE RANDOM ANYWAY DUE TO INHERENT RANDOMNESS
        self.ev = np.zeros(shape=(length,1))
        # Loop if we get all zeros
        while not self.ev.any():
            self.ev = np.random.randint(255, size=(length, 1))
        # Scale to have unit length (convert to integer values?)
        self.ev = self.ev / np.linalg.norm(self.ev)
        # Define placeholder for covariance matrix
        self.cov = np.zeros(shape=(length, length))
        # Define scaling factor as 1/sqrt(length)
        self.scaler = np.sqrt(length) / length

    def iterate(self, power=1, cov=None):
        """One pass of iteration."""
        # If a covariance is passed use to update
        if cov is not None:
            self.load_covariance(cov)
        # Check cov is not all zero - if all 0 you get nan
        if self.cov.any():
            # Times covariance by working eigenvector
            if power == 1:
                # Speed up for single power
                self.ev = np.matmul(self.cov, self.ev)
            else:
                self.ev = np.matmul(np.power(self.cov, power), self.ev)
            # Scale to have unit length (convert to integer values?)
            self.ev = self.ev / np.linalg.norm(self.ev)
        return self.ev.copy()

    @property
    def eigenvector(self):
        """Return the top eigenvector."""
        return self.ev.copy()

    @property
    def eigenvalue(self):
        """Return associated eigenvalue."""
        top_1 = np.matmul(self.ev.T, self.cov)
        bottom = np.matmul(self.ev.T, self.ev)
        r = np.matmul(top_1, self.ev) / bottom
        return r

    @property
    def feature(self):
        """Return eigenvector scaled to ternary space."""
        return self.ev*self.scaler

    def load_covariance(self, cov):
        """Update the covariance matrix."""
        self.cov = cov
        # Put here an update of an existing matrix?


---
# Reference Stuff

In [ ]:
# Version with variable bit depth
class CovarianceUnit:
    """Variation where the mean is assumed to be 0."""
    
    def __init__(self, size, bit_depth=8):
        """Initialise.

        Args:
            size: integer setting the 1D size of an input.
            bit_depth: number of bits to store representations (multiple of 8) - max 64
        """
        self.size = size
        self.bit_depth = bit_depth
        self.max_value = 2**bit_depth
        # Set bit_depth modulo 8
        if bit_depth > 64: bit_depth = 64
        if bit_depth < 8: bit_depth = 8
        stages = 64//bit_depth
        # Set 
        # Initialise Square Sum
        self.square_sum = np.ones(shape=(size, size, stages), dtype=)
        # Set to halfway between 0 and bit depth (128 for 8-bit)
        self.square_sum = 2**(bit_depth-1)*self.square_sum
        # Define counter for each stage
        self.stage_counter = np.zeros(stages)
        
    def update(self, x):
        """Add a data point x.
        
        This will involve a recursive check.

        x is a 1D numpy array of length 'size'.
        """
        self.stage_counters[0] += 1
        self.square_sum[:, :, 0] += np.dot(x, x.T)
        # If the count reaches a threshold, can we divide the sums
        if self.stage_counters[0] >= (2**bit_depth)-1:
            # Subtract estimate from next stage
            square_sum_dash = self.square_sum[:, :, 0] - self.square_sum[:, :, 1]
            # PBT self.square_sum[:, :, 0] - BUT WILL THIS BE TERNARY? 
            rand_ints = np.random.randint(
                self.max_value, size=(self.size, self.size)
            )
            signs = np.signs(square_sum_dash)
            pbt_output = np.where(self.square_sum[:, :, 0], rand_ints, 1, 0)
            # Add to next stage
            self.square_sum[:, :, 1] += pbt_output
            # Increment next stage counter
            self.stage_counters[1] += 1
            # Reset the previous counter
            self.stage_counters[0] = 0
            if self.stage_counters[1] >= (2**bit_depth)-1:
                

    @property
    def covariance(self):
        """Compute covariance when requested."""
        # Only return is count is 1 or more
        if self.count:
            cov = self.square_sum / self.count
        else:
            cov = self.square_sum
        return cov
